# 실습 2. 서울시 상권별 월평균 매출액 예측  
1. 데이터: 서울시 우리마을 가게 상권배후지추정매출 (2014년 ~ 2020년 2분기)
2. 예측: 상권 종목 별 월평균 매출액 예측 (KRW)
3. 특징: 시계열 데이터 변환
4. 목표: 분기별 예측 R2 80% 이상

In [3]:
import pandas as pd
import numpy as np
import os
forders = os.listdir('data/상권배후지추정매출')
df_all_years = pd.DataFrame()
for i in range(0,len(forders)):
    path = 'data/상권배후지추정매출/'+forders[i]
    df= pd.read_csv(path,encoding='cp949',thousands = ',') 
    df_all_years = pd.concat([df_all_years, df])
df_all_years.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2014,1,A,골목상권,1000522,목동로9길,CS200041,사진관,161604397,3021,...,63,1450,1435,97,683,679,674,441,311,5
1,2014,1,A,골목상권,1000580,공항대로8길,CS100007,치킨전문점,352909128,14422,...,5746,8280,5597,256,3970,4688,2750,1707,510,19
2,2014,1,A,골목상권,1000507,가로공원로58길,CS200028,미용실,174625276,5695,...,289,2639,3044,130,1234,1561,1440,925,391,36
3,2014,1,A,골목상권,1000556,강서로18길,CS300035,인테리어,158530192,1571,...,30,395,1158,20,127,662,366,286,92,21
4,2014,1,A,골목상권,1000509,곰달래로2길,CS100007,치킨전문점,238151311,9718,...,4157,5763,3573,171,2165,3484,2160,1038,316,11


In [4]:
df_all_years.shape

(1434810, 80)

In [7]:
df_all_years.isna().sum()

기준_년_코드            0
기준_분기_코드           0
상권_구분_코드           0
상권_구분_코드_명         0
상권_코드              0
                  ..
연령대_30_매출_건수       0
연령대_40_매출_건수       0
연령대_50_매출_건수       0
연령대_60_이상_매출_건수    0
점포수                0
Length: 80, dtype: int64

In [17]:
df_all_years['서비스_업종_코드'].nunique()

82

### 중복 칼럼 제거

In [18]:
df_all_years.drop(['상권_구분_코드','상권_구분_코드_명','상권_코드_명','서비스_업종_코드_명'],axis =1, inplace=True)
df_all_years.head()

,기준_년_코드,기준_분기_코드,상권_코드,서비스_업종_코드,당월_매출_금액,당월_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2014,1,1000522,CS200041,161604397,3021,72,28,11,25,...,63,1450,1435,97,683,679,674,441,311,5
1,2014,1,1000580,CS100007,352909128,14422,69,31,12,13,...,5746,8280,5597,256,3970,4688,2750,1707,510,19
2,2014,1,1000507,CS200028,174625276,5695,65,35,16,8,...,289,2639,3044,130,1234,1561,1440,925,391,36
3,2014,1,1000556,CS300035,158530192,1571,88,13,9,8,...,30,395,1158,20,127,662,366,286,92,21
4,2014,1,1000509,CS100007,238151311,9718,70,30,13,16,...,4157,5763,3573,171,2165,3484,2160,1038,316,11
